In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Cloud Deploy Vertex AI Model Deployer开始

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_registry/get_started_with_vertex_ai_deployer.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_registry/get_started_with_vertex_ai_deployer.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_registry/get_started_with_vertex_ai_deployer.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

请注意：此笔记本在以下环境中进行了测试：

* Python版本= 3.9

## 概览

在模型经过训练、验证并添加到模型注册表后，它就准备好部署了。在部署模型之前，您可能需要经过一个持续部署过程来测试和验证模型。

本教程演示了如何使用 Cloud Deploy 自定义目标将 Vertex AI 模型部署到一个端点，这允许您指定一个 Vertex AI 端点作为运行时环境来部署您的模型。

了解更多关于 [Cloud Deploy](https://cloud.google.com/deploy) 和 [Vertex AI](https://cloud.google.com/vertex-ai)。

### 目标

在本教程中，您将学习如何使用云部署自定义目标将 Vertex AI 模型部署到端点：

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex AI 模型注册表
- Vertex AI 端点
- 云部署

执行的步骤包括：

- 将 Vertex AI 模型导入模型注册表，并创建一个端点，用于部署模型。
- 定义一个 Cloud Deploy 交付流水线，为 Vertex AI 定制目标类型，并创建一个目标。
- 创建一个 Cloud Deploy 发布并展开部署 Vertex AI 模型到目标端点。

数据集

用于预训练模型的数据集是波士顿房屋数据集，包含由美国人口普查局收集的有关波士顿Mass地区住房的信息。

### 费用

本教程使用 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage
* Cloud Deploy

了解有关 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage 定价](https://cloud.google.com/storage/pricing) 和 [Cloud Deploy 定价](https://cloud.google.com/cloud-deploy/pricing) 的信息，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

<div class="alert alert-block alert-warning">
<b>⚠️ 这不是一个官方支持的谷歌产品，也不受谷歌云支持合同的保护。
如需报告谷歌云产品中的错误或请求功能，请联系谷歌云支持⚠️</b>
</div>

安装

安装以下所需的软件包以执行此笔记本。

{待办事项:建议使用每个软件包的最新主要GA版本;即，--upgrade}

In [ ]:
# Install the packages
import os

if not os.getenv("IS_TESTING"):
    USER = "--user"
else:
    USER = ""
! pip3 install {USER} --upgrade google-cloud-aiplatform

只有协作: 取消以下单元格的注释以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

##开始前

###设置你的Google Cloud项目

**无论您使用哪种笔记本环境，以下步骤都是必须的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用额度可用于计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用API](https://console.cloud.google.com/flows/enableapi?apiid=clouddeploy.googleapis.com,containeranalysis.googleapis.com,compute.googleapis.com,aiplatform.googleapis.com)。

4. 如果您正在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行`gcloud config list`。
* 运行`gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. 顶点 AI 工作台**
* 无需操作，因为您已经通过身份验证。

2. 本地JupyterLab实例，请取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

将Cloud Storage权限授予您的服务帐户，请参阅https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建云存储桶

创建一个存储桶来存储诸如数据集等中间产物。

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 服务帐户

**如果您不知道您的服务帐户**，请尝试使用`gcloud`命令在下面执行第二个单元格来获取您的服务帐户。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### 为服务账号设置权限

**重要提示**：在运行以下代码之前，请确保您已经通过服务账号进行了身份验证，并检查其是否具有必要的权限。请参阅[这里](https://cloud.google.com/iam/docs/manage-access-service-accounts#before_you_begin)的 `开始之前` 部分了解更多详细信息。

要使用Cloud Deploy，您需要在您的服务账号上设置以下角色：

* `roles/clouddeploy.jobRunner`，这是创建Cloud Deploy资源、在流水线中创建发布和部署所必需的。

* `roles/clouddeploy.developer`，用于访问Cloud Deploy资源的权限。

* `roles/clouddeploy.operator`，用于管理Cloud Deploy流水线、目标资源、发布、部署和作业运行的权限。

* `roles/clouddeploy.customTargetTypeAdmin`，用于完全控制Cloud Deploy自定义目标类型的权限。

* `roles/containeranalysis.notes.editor`，用于让自定义镜像在发布过程中运行 Artifact Analysis。

* `roles/aiplatform.user`，用于自定义镜像创建和导入模型、访问模型信息，并进行模型部署。

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role="roles/clouddeploy.jobRunner"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role="roles/clouddeploy.developer"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role="roles/clouddeploy.operator"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role="roles/clouddeploy.customTargetTypeAdmin"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role="roles/containeranalysis.notes.editor"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT} \
    --role="roles/aiplatform.user"

### 导入库

In [ ]:
from google.cloud import aiplatform as vertex_ai

### 设置变量

在下面设置你在本教程中使用的模型和服务图像。

In [ ]:
MODEL_URI = "gs://cloud-samples-data/vertex-ai/model-deployment/models/boston/model"
DEPLOY_IMAGE = (
    "us-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-cpu.nightly:latest"
)

### 初始化用于 Python 的 Vertex AI SDK

为您的项目初始化用于 Python 的 Vertex AI SDK。

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

Cloud Deploy Vertex AI Deployer是Cloud Deploy的自定义目标。

Cloud Deploy是一项托管的持续交付服务。用户可以使用Cloud Deploy来定义交付流水线，并配置一系列有序的目标。通过自定义目标，用户可以将部署到除了支持的运行时之外的其他系统，包括Vertex AI端点。

使用Vertex AI，一旦定义了Vertex AI目标端点，用户可以创建一个发布，与模型的特定版本相关联。然后他们可以创建一个rollout，这是将模型发布部署到流水线序列中特定目标端点的操作。最后，用户可以将此发布推广到序列中的下一个目标。

### 将模型注册到Vertex AI模型注册表

您将模型上传为`模型注册表`中`Model`资源的一个版本。至少，您需要指定显示名称、模型的Cloud Bucket位置和要使用的预测图像。

在本教程中，您将使用来自Vertex AI云示例存储桶的预训练模型。对于Docker镜像，您将使用Cloud AI优化版本的tensorflow：[tf_opt](https://cloud.google.com/vertex-ai/docs/predictions/optimized-tensorflow-runtime)。

In [ ]:
registered_model = vertex_ai.Model.upload(
    display_name="model_to_deploy",
    artifact_uri=MODEL_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(registered_model)

### 创建一个 Vertex AI 端点

您可以使用 `Endpoint.create()` 方法来创建一个 `Endpoint` 资源。至少，您需要指定端点的显示名称。

In [ ]:
endpoint = vertex_ai.Endpoint.create(
    display_name="target_endpoint",
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint)

### 创建交付管道，目标和自定义目标类型

当您使用Vertex AI Deployer部署已注册模型时，您将涵盖以下步骤：

- 您定义一个自定义操作，类似于部署挂钩，并在skaffold.yaml文件中定义。

- 您定义一个自定义目标类型，这是一个标识用于此类型目标的自定义操作的Cloud Deploy资源。在您的情况下，是一个Vertex AI Endpoint。

- 您为自定义目标设置目标定义与任何目标类型相同，只是包括一些属性。

一旦您拥有所有必要的组件，您将设置一个Cloud Deploy交付管道，引用配置的目标。您可以通过引用目标来部署模型，充分利用Cloud Deploy的功能，如推广、审批和回滚。如果要了解更多关于自定义目标以及自定义目标如何在Cloud Deploy中工作的信息，请查看[官方文档](https://cloud.google.com/deploy/docs/custom-targets)。

要完成所有这些步骤，您可以使用[vertex-ai](https://github.com/GoogleCloudPlatform/cloud-deploy-samples/tree/main/custom-targets/vertex-ai)目录中的`build_and_register.sh`脚本。该脚本可用于构建Vertex AI模型部署器镜像并注册引用该镜像的Cloud Deploy自定义目标类型。

接下来，您可以克隆Cloud Deploy示例存储库，并将当前目录设置为vertex AI示例快速入门文件夹。

In [ ]:
!git clone https://github.com/googlecloudplatform/cloud-deploy-samples.git
%cd cloud-deploy-samples/custom-targets/vertex-ai/quickstart

运行构建图像并注册自定义目标类型的脚本。

In [ ]:
!../build_and_register.sh -p $PROJECT_ID -r $REGION

运行以下命令，将Cloud Deploy和Skaffold配置值中的占位符替换为实际图像。

In [ ]:
endpoint_id = endpoint.name

!./replace_variables.sh -p $PROJECT_ID -r $REGION -e $endpoint_id

最后，应用在`clouddeploy.yaml`中定义的云部署配置。

In [ ]:
! gcloud deploy apply --file=clouddeploy.yaml --project=$PROJECT_ID --region=$REGION

创建一个发布和部署计划

为“configuration”中定义的配置目录创建一个云部署发布。请注意，配置文件中包含与您的端点相关的部署设置，包括机器类型、副本数量等。这将自动创建一个部署计划，将第一个模型版本部署到目标端点。

要创建一个云部署发布，您必须指定

* `--source`命令行标志，指示gcloud在工作目录中相对于哪里查找配置文件。

* `--deploy-parameters`标志，用于向自定义部署程序提供执行部署所需的额外参数。

在这里，您提供给自定义部署程序两个参数：

* `customTarget/vertexAIModel`，指示要部署的模型的完整资源名称

* `--delivery-pipeline`，是发布将被创建的交付管道的名称，并且通过`--project`和`--region`分别指定管道的项目和区域。

In [ ]:
model_id = registered_model.name
deploy_params = f'customTarget/vertexAIModel=projects/{PROJECT_ID}/locations/{REGION}/models/{model_id}'

! gcloud deploy releases create release-0001 \
    --delivery-pipeline=vertex-ai-cloud-deploy-pipeline \
    --project=$PROJECT_ID \
    --region=$REGION \
    --source=configuration \
    --deploy-parameters=$deploy_params

### 监视发布进度

要检查发布详情，请运行以下命令。

In [ ]:
!gcloud deploy releases describe release-0001 --delivery-pipeline=vertex-ai-cloud-deploy-pipeline --project=$PROJECT_ID --region=$REGION

运行此命令以筛选仅发布的渲染状态。

In [ ]:
!gcloud deploy releases describe release-0001 --delivery-pipeline=vertex-ai-cloud-deploy-pipeline --project=$PROJECT_ID --region=$REGION --format "(renderState)"

<div class="alert alert-block alert-warning">
<b>
⚠️ 模型完全部署可能需要最多15分钟。 ⚠️
</b>
</div>

### 监控部署状态

在您的项目的[Cloud部署UI](https://cloud.google.com/deploy)中，点击`vertex-ai-cloud-deploy-pipeline`交付流水线。在这里，您可以看到已创建的发布和针对发布的dev目标的部署。

您也可以使用以下命令描述已创建的部署。

In [ ]:
! gcloud deploy rollouts describe release-0001-to-prod-endpoint-0001 --release=release-0001 --delivery-pipeline=vertex-ai-cloud-deploy-pipeline --project=$PROJECT_ID --region=$REGION

检查部署模型的端点

在部署完成后，您可以使用`gcloud`检查端点的部署模型和流量分配。

In [ ]:
! gcloud ai endpoints describe $endpoint_id --region $REGION --project $PROJECT_ID

### 检查部署模型中的别名

通过查询部署后的操作监控发布。

In [ ]:
!gcloud deploy rollouts describe release-0001-to-prod-endpoint-0001 --release=release-0001 --delivery-pipeline=vertex-ai-cloud-deploy-pipeline --project=$PROJECT_ID --region=$REGION --format "(phases[0].deploymentJobs.postdeployJob)"

在部署后作业成功之后，您可以检查已部署的模型并查看其当前分配的别名。 应该分配`prod`和`champion`这两个别名。 这些别名将帮助您管理模型的交付过程。

In [ ]:
! gcloud ai models describe $model_id --region $REGION --project $PROJECT_ID --format "(versionAliases)"

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以按照以下步骤删除本教程中创建的各个资源。

In [ ]:
delete_endpoint = True
delete_model = True
delete_cloud_deploy = True

if delete_endpoint or os.getenv("IS_TESTING"):
    endpoint_list = vertex_ai.Endpoint.list(filter='display_name="target_endpoint"')
    for endpoint in endpoint_list:
        endpoint.delete(force=True)

if delete_model or os.getenv("IS_TESTING"):
    model_list = vertex_ai.Model.list(filter='display_name="model_to_deploy"')
    for model in model_list:
        model.delete()

if delete_model or os.getenv("IS_TESTING"):
    ! gcloud deploy delete --file=clouddeploy.yaml --force --project=$PROJECT_ID --region=$REGION